In [1]:
from pathlib import Path
import sys

home_path = str(Path.cwd().parent)
sys.path.append(home_path)

sys.path.append('/eos/home-i03/m/morwat/.local/lib/python3.9/site-packages/')

In [2]:
from aper_package.interactive_tool import InteractiveTool

In [4]:
tool = InteractiveTool(initial_path=home_path+'/test_data', spark=spark)
tool.show()